<a href="https://colab.research.google.com/github/Hannauer/mineracao_e_aprendizado_nlp/blob/main/01_eda_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install contractions

     |████████████████████████████████| 245kB 3.9MB/s 
     |████████████████████████████████| 317kB 17.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81710 sha256=17f8331414a267b80ff7b7cfad6fd12adc28943d30a6dcf6a8812debebc21034
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [48]:
import pandas as pd
import os
import numpy as np
import nltk
import string
import fasttext
import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import SnowballStemmer
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)

In [16]:

df = pd.read_csv('/content/IMDB_Dataset.csv', delimiter=',', error_bad_lines=False)


b'Skipping line 4049: expected 2 fields, saw 8\nSkipping line 4846: expected 2 fields, saw 4\n'


In [26]:
print(df.shape)
df.head()


(54850, 4)


,review,sentiment,no_contract,review_description_str
0,One of the other reviewers has mentioned that ...,positive,"[One, of, the, other, reviewers, has, mentione...",One of the other reviewers has mentioned that ...
1,A wonderful little production. The filming...,positive,"[A, wonderful, little, production., The, filmi...",A wonderful little production. The filming tec...
2,I thought this was a wonderful way to spend ti...,positive,"[I, thought, this, was, a, wonderful, way, to,...",I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,"[Basically, there is, a, family, where, a, lit...",Basically there is a family where a little boy...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[Petter, Mattei's, ""Love, in, the, Time, of, M...","Petter Mattei's ""Love in the Time of Money"" is..."


In [20]:
#verificando se existem valores nulos na targe
df['review'].isna().sum()

0

In [21]:
#verificando as categorias e suas volumetrias
df.sentiment.value_counts()

positive    27512
negative    27338
Name: sentiment, dtype: int64

In [19]:
#retirando as contrações 
df['no_contract'] = df['review'].apply(lambda x: [contractions.fix(word) for word in x.split()])
df.head()

,review,sentiment,no_contract
0,One of the other reviewers has mentioned that ...,positive,"[One, of, the, other, reviewers, has, mentione..."
1,A wonderful little production. The filming...,positive,"[A, wonderful, little, production., The, filmi..."
2,I thought this was a wonderful way to spend ti...,positive,"[I, thought, this, was, a, wonderful, way, to,..."
3,Basically there's a family where a little boy ...,negative,"[Basically, there is, a, family, where, a, lit..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[Petter, Mattei's, ""Love, in, the, Time, of, M..."


In [22]:
#transformando as descrições em linhas 
df['review_description_str'] = [' '.join(map(str, l)) for l in df['no_contract']]
df.head()

,review,sentiment,no_contract,review_description_str
0,One of the other reviewers has mentioned that ...,positive,"[One, of, the, other, reviewers, has, mentione...",One of the other reviewers has mentioned that ...
1,A wonderful little production. The filming...,positive,"[A, wonderful, little, production., The, filmi...",A wonderful little production. The filming tec...
2,I thought this was a wonderful way to spend ti...,positive,"[I, thought, this, was, a, wonderful, way, to,...",I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,"[Basically, there is, a, family, where, a, lit...",Basically there is a family where a little boy...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[Petter, Mattei's, ""Love, in, the, Time, of, M...","Petter Mattei's ""Love in the Time of Money"" is..."


In [ ]:
#dowload do db de pontuação
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [30]:
#tokenizando as palavras
df['tokenized'] = df['review_description_str'].apply(word_tokenize)
df.head()

,review,sentiment,no_contract,review_description_str,tokenized
0,One of the other reviewers has mentioned that ...,positive,"[One, of, the, other, reviewers, has, mentione...",One of the other reviewers has mentioned that ...,"[One, of, the, other, reviewers, has, mentione..."
1,A wonderful little production. The filming...,positive,"[A, wonderful, little, production., The, filmi...",A wonderful little production. The filming tec...,"[A, wonderful, little, production, ., The, fil..."
2,I thought this was a wonderful way to spend ti...,positive,"[I, thought, this, was, a, wonderful, way, to,...",I thought this was a wonderful way to spend ti...,"[I, thought, this, was, a, wonderful, way, to,..."
3,Basically there's a family where a little boy ...,negative,"[Basically, there is, a, family, where, a, lit...",Basically there is a family where a little boy...,"[Basically, there, is, a, family, where, a, li..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[Petter, Mattei's, ""Love, in, the, Time, of, M...","Petter Mattei's ""Love in the Time of Money"" is...","[Petter, Mattei, 's, ``, Love, in, the, Time, ..."


In [31]:
#convertendo todos os caracter para lower case
df['lower'] = df['tokenized'].apply(lambda x: [word.lower() for word in x])
df.head()

,review,sentiment,no_contract,review_description_str,tokenized,lower
0,One of the other reviewers has mentioned that ...,positive,"[One, of, the, other, reviewers, has, mentione...",One of the other reviewers has mentioned that ...,"[One, of, the, other, reviewers, has, mentione...","[one, of, the, other, reviewers, has, mentione..."
1,A wonderful little production. The filming...,positive,"[A, wonderful, little, production., The, filmi...",A wonderful little production. The filming tec...,"[A, wonderful, little, production, ., The, fil...","[a, wonderful, little, production, ., the, fil..."
2,I thought this was a wonderful way to spend ti...,positive,"[I, thought, this, was, a, wonderful, way, to,...",I thought this was a wonderful way to spend ti...,"[I, thought, this, was, a, wonderful, way, to,...","[i, thought, this, was, a, wonderful, way, to,..."
3,Basically there's a family where a little boy ...,negative,"[Basically, there is, a, family, where, a, lit...",Basically there is a family where a little boy...,"[Basically, there, is, a, family, where, a, li...","[basically, there, is, a, family, where, a, li..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[Petter, Mattei's, ""Love, in, the, Time, of, M...","Petter Mattei's ""Love in the Time of Money"" is...","[Petter, Mattei, 's, ``, Love, in, the, Time, ...","[petter, mattei, 's, ``, love, in, the, time, ..."


In [32]:
#removendo pontuação
punc = string.punctuation
df['no_punc'] = df['lower'].apply(lambda x: [word for word in x if word not in punc])
df.head()

,review,sentiment,no_contract,review_description_str,tokenized,lower,no_punc
0,One of the other reviewers has mentioned that ...,positive,"[One, of, the, other, reviewers, has, mentione...",One of the other reviewers has mentioned that ...,"[One, of, the, other, reviewers, has, mentione...","[one, of, the, other, reviewers, has, mentione...","[one, of, the, other, reviewers, has, mentione..."
1,A wonderful little production. The filming...,positive,"[A, wonderful, little, production., The, filmi...",A wonderful little production. The filming tec...,"[A, wonderful, little, production, ., The, fil...","[a, wonderful, little, production, ., the, fil...","[a, wonderful, little, production, the, filmin..."
2,I thought this was a wonderful way to spend ti...,positive,"[I, thought, this, was, a, wonderful, way, to,...",I thought this was a wonderful way to spend ti...,"[I, thought, this, was, a, wonderful, way, to,...","[i, thought, this, was, a, wonderful, way, to,...","[i, thought, this, was, a, wonderful, way, to,..."
3,Basically there's a family where a little boy ...,negative,"[Basically, there is, a, family, where, a, lit...",Basically there is a family where a little boy...,"[Basically, there, is, a, family, where, a, li...","[basically, there, is, a, family, where, a, li...","[basically, there, is, a, family, where, a, li..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[Petter, Mattei's, ""Love, in, the, Time, of, M...","Petter Mattei's ""Love in the Time of Money"" is...","[Petter, Mattei, 's, ``, Love, in, the, Time, ...","[petter, mattei, 's, ``, love, in, the, time, ...","[petter, mattei, 's, ``, love, in, the, time, ..."


In [34]:
#download do dic de stop words
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [35]:
#removendo stop words
stop_words = set(stopwords.words('english'))
df['stopwords_removed'] = df['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])
df.head()

,review,sentiment,no_contract,review_description_str,tokenized,lower,no_punc,stopwords_removed
0,One of the other reviewers has mentioned that ...,positive,"[One, of, the, other, reviewers, has, mentione...",One of the other reviewers has mentioned that ...,"[One, of, the, other, reviewers, has, mentione...","[one, of, the, other, reviewers, has, mentione...","[one, of, the, other, reviewers, has, mentione...","[one, reviewers, mentioned, watching, 1, oz, e..."
1,A wonderful little production. The filming...,positive,"[A, wonderful, little, production., The, filmi...",A wonderful little production. The filming tec...,"[A, wonderful, little, production, ., The, fil...","[a, wonderful, little, production, ., the, fil...","[a, wonderful, little, production, the, filmin...","[wonderful, little, production, filming, techn..."
2,I thought this was a wonderful way to spend ti...,positive,"[I, thought, this, was, a, wonderful, way, to,...",I thought this was a wonderful way to spend ti...,"[I, thought, this, was, a, wonderful, way, to,...","[i, thought, this, was, a, wonderful, way, to,...","[i, thought, this, was, a, wonderful, way, to,...","[thought, wonderful, way, spend, time, hot, su..."
3,Basically there's a family where a little boy ...,negative,"[Basically, there is, a, family, where, a, lit...",Basically there is a family where a little boy...,"[Basically, there, is, a, family, where, a, li...","[basically, there, is, a, family, where, a, li...","[basically, there, is, a, family, where, a, li...","[basically, family, little, boy, jake, thinks,..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[Petter, Mattei's, ""Love, in, the, Time, of, M...","Petter Mattei's ""Love in the Time of Money"" is...","[Petter, Mattei, 's, ``, Love, in, the, Time, ...","[petter, mattei, 's, ``, love, in, the, time, ...","[petter, mattei, 's, ``, love, in, the, time, ...","[petter, mattei, 's, ``, love, time, money, ''..."


In [45]:
#aplicando o stemmer nas palavras
stemmer = SnowballStemmer(language='english')
df['stemmed'] = df['stopwords_removed'].apply(lambda x: [stemmer.stem(word) for word in x])
df.head()



,review,sentiment,no_contract,review_description_str,tokenized,lower,no_punc,stopwords_removed,stemmed
0,One of the other reviewers has mentioned that ...,positive,"[One, of, the, other, reviewers, has, mentione...",One of the other reviewers has mentioned that ...,"[One, of, the, other, reviewers, has, mentione...","[one, of, the, other, reviewers, has, mentione...","[one, of, the, other, reviewers, has, mentione...","[one, reviewers, mentioned, watching, 1, oz, e...","[one, review, mention, watch, 1, oz, episod, h..."
1,A wonderful little production. The filming...,positive,"[A, wonderful, little, production., The, filmi...",A wonderful little production. The filming tec...,"[A, wonderful, little, production, ., The, fil...","[a, wonderful, little, production, ., the, fil...","[a, wonderful, little, production, the, filmin...","[wonderful, little, production, filming, techn...","[wonder, littl, product, film, techniqu, unass..."
2,I thought this was a wonderful way to spend ti...,positive,"[I, thought, this, was, a, wonderful, way, to,...",I thought this was a wonderful way to spend ti...,"[I, thought, this, was, a, wonderful, way, to,...","[i, thought, this, was, a, wonderful, way, to,...","[i, thought, this, was, a, wonderful, way, to,...","[thought, wonderful, way, spend, time, hot, su...","[thought, wonder, way, spend, time, hot, summe..."
3,Basically there's a family where a little boy ...,negative,"[Basically, there is, a, family, where, a, lit...",Basically there is a family where a little boy...,"[Basically, there, is, a, family, where, a, li...","[basically, there, is, a, family, where, a, li...","[basically, there, is, a, family, where, a, li...","[basically, family, little, boy, jake, thinks,...","[basic, famili, littl, boy, jake, think, zombi..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[Petter, Mattei's, ""Love, in, the, Time, of, M...","Petter Mattei's ""Love in the Time of Money"" is...","[Petter, Mattei, 's, ``, Love, in, the, Time, ...","[petter, mattei, 's, ``, love, in, the, time, ...","[petter, mattei, 's, ``, love, in, the, time, ...","[petter, mattei, 's, ``, love, time, money, ''...","[petter, mattei, 's, ``, love, time, money, ''..."


In [51]:
#gerando coeficiente de sentimento
df_processed = df[['stemmed', 'sentiment']]

df_processed['stemmed_str'] = [' '.join(map(str,l)) for l in df['stemmed']]

df_processed['sentiment_score'] = df_processed['stemmed_str'].apply(lambda x: TextBlob(x).sentiment.polarity)
df_processed.head()

,stemmed,sentiment,stemmed_str,sentiment_score
0,"[one, review, mention, watch, 1, oz, episod, hook, right, exact, happen, first, thing, struck, o...",positive,one review mention watch 1 oz episod hook right exact happen first thing struck oz brutal unflin...,0.025782
1,"[wonder, littl, product, film, techniqu, unassuming-, old-time-bbc, fashion, give, comfort, some...",positive,wonder littl product film techniqu unassuming- old-time-bbc fashion give comfort sometim discomf...,0.215476
2,"[thought, wonder, way, spend, time, hot, summer, weekend, sit, air, condit, theater, watch, ligh...",positive,thought wonder way spend time hot summer weekend sit air condit theater watch light-heart comedi...,0.372619
3,"[basic, famili, littl, boy, jake, think, zombi, closet, parent, fight, time, movi, slower, soap,...",negative,basic famili littl boy jake think zombi closet parent fight time movi slower soap opera ... sudd...,0.056250
4,"[petter, mattei, 's, ``, love, time, money, '', visual, stun, film, watch, mr., mattei, offer, u...",positive,petter mattei 's `` love time money '' visual stun film watch mr. mattei offer us vivid portrait...,0.193974


In [52]:
#salvando o novo data frame
df.to_pickle('processed_imdb.pkl')